In [1]:
!pip install -U langchain_huggingface faiss-cpu faiss-gpu beautifulsoup4 requests transformers peft accelerate bitsandbytes langchain_community langgraph pymupdf langchain_openai PyPDF2 trl torch torchvision


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 54.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.0/186.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 103.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 21.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.4/142.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 78.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.7 MB/s

In [2]:
from langchain.agents import tool

from langchain_community.document_loaders import WebBaseLoader

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.agents import tool

import os
import nest_asyncio
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings  # Updated import
from langchain.schema import Document

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
import torchvision
torchvision.disable_beta_transforms_warning()

import bs4
from langchain_community.document_loaders import WebBaseLoader

In [3]:
def load_pdfs_from_folder(folder_path):
    pdf_documents = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'rb') as f:
                reader = PdfReader(f)
                text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
                pdf_documents.append(Document(page_content=text))  # Convert to LangChain Document
    return pdf_documents

# Improved text splitting function
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    split_documents = text_splitter.split_documents(documents)
    return split_documents

# Function to load and process documents for RAG (only PDFs now)
def load_and_process_documents(pdf_folder):
    # Load PDFs from folder
    pdf_documents = load_pdfs_from_folder(pdf_folder)

    # Split documents into manageable chunks
    split_documents_list = split_documents(pdf_documents)

    return split_documents_list


In [4]:
# # Load documents (only PDFs)
# folder_path = "/kaggle/input/therapist-docs/RAG docs"
# documents = load_and_process_documents(folder_path)

# Define embedding model (using the updated HuggingFaceEmbeddings)
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

# # Create FAISS vector store
# vector_db = FAISS.from_documents(documents, embeddings)

# vector_db.save_local("/kaggle/working/FAISS")

# # Define retriever with improved parameters for better retrieval
# retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={'k': 4})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
# relevant_webpages = [
#     "https://beckinstitute.org/beck-institute-role-play-transcript-abe-therapy-session-2",
#     "https://beckinstitute.org/wp-content/uploads/2021/06/BB3-Session-2-Annotated-Transcript.pdf",
#     "https://www.cci.health.wa.gov.au/Training/Demonstration-Videos",
#     "https://www.psychotherapy.net/data/fe/file/Somme_Couns_transcript.pdf",
#     "https://kmb.camh.ca/ggtu/Documents/cbt-clinical-simulation-transcript.pdf",
#     "https://www.mirecc.va.gov/visn16/docs/therapists_guide_to_brief_cbtmanual.pdf",
#     "https://www.cci.health.wa.gov.au/Resources/Looking-After-Yourself"
# ]

# bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

# for webpage in relevant_webpages:
#     try:
#         # Load webpage content
#         response = requests.get(webpage, verify=False)  # Disable SSL verification (use cautiously)
#         print(response.status_code)
#         loader = WebBaseLoader(web_paths=[webpage], bs_kwargs={"parse_only": bs4_strainer})
#         docs = loader.load()

#         # Split into chunks
#         text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
#         split_documents = text_splitter.split_documents(docs)

#         # Add to FAISS index
#         vector_db.add_documents(split_documents)

#     except Exception as e:
#         print(f"Error loading {webpage}: {e}")

# vector_db.save_local("/kaggle/working/FAISS")
# retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={'k': 4})


In [6]:
# Load FAISS index
### RAG functions
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2') #model_kwargs={"device": "cpu"}
faiss_index = FAISS.load_local("/kaggle/input/faiss-index-2", embeddings=embeddings,allow_dangerous_deserialization = True)

retriever = faiss_index.as_retriever(search_type="similarity", search_kwargs={'k': 4})


Tool Function	Recommended Type	Why?
RAG for retrieving therapy-related responses-Structured Tool: Outputs formatted data (retrieved documents)
Memory tracking (checking distress, session booking in JSON)-Subclass-Based Tool: Needs custom logic for storing structured session data
Suicidal Detection & Notification-Subclass-Based Tool: Requires specialized logic to trigger alerts
Guardrails (Filtering harmful responses)-Normal Tool: Simple validation of responses
Calming Music Recommendation-Normal Tool: Simple retrieval of URLs

In [7]:
import json
from langchain.tools import BaseTool
from typing import Dict

# Store per-session distress data
distress_memory_store: Dict[str, Dict] = {}

class MemoryTrackingTool(BaseTool):
    name: str = "distress_memory_tracker"  # ✅ Must be annotated!
    description: str = "Detects distress in user input. Format: 'session_id::message'."

    def _run(self, input: str) -> str:
        try:
            session_id, user_input = input.split("::", 1)
        except ValueError:
            return "Invalid format. Use 'session_id::message'"

        # Track memory per session
        if session_id not in distress_memory_store:
            distress_memory_store[session_id] = {"distress_count": 0, "session_booked": False}

        session_data = distress_memory_store[session_id]

        distress_keywords = ["hopeless", "helpless", "overwhelmed", "exhausted"]
        session_keywords = ["book session", "schedule appointment"]

        if any(word in user_input.lower() for word in distress_keywords):
            session_data["distress_count"] += 1
            if session_data["distress_count"] >= 3 and not session_data["session_booked"]:
                return "You've mentioned distress multiple times. Would you like to book a session with a counselor?"

        if any(word in user_input.lower() for word in session_keywords):
            session_data["session_booked"] = True
            with open("session_bookings.json", "a") as f:
                json.dump({session_id: "booked"}, f)
                f.write("\n")
            return "Your session has been booked."

        return "Monitoring your responses. Let me know if you'd like to talk more or book a session."

memory_tool = MemoryTrackingTool()

In [8]:
class SuicidalDetectionTool(BaseTool):
    name: str = "suicidal_detection"
    description: str = "Detects suicidal thoughts and triggers notifications."

    def _run(self, user_input: str):
        """Detects suicidal intent and triggers intervention if needed."""
        suicidal_keywords = ["I want to die", "end my life", "can't go on", "no point in living"]

        if any(phrase in user_input.lower() for phrase in suicidal_keywords):
            return """I'm really sorry to hear that you're feeling this way. You are not alone. 
    Please consider reaching out to a trusted friend, family member, or professional.
    
    If you are in immediate danger, please contact a crisis helpline:
    - Singapore: Samaritans of Singapore (SOS) - 1767
    - USA: National Suicide Prevention Lifeline - 988
    - UK: Samaritans - 116 123
    """

        return "No suicidal intent detected."

suicidal_tool = SuicidalDetectionTool()


In [9]:
from langchain.tools import Tool

def recommend_calming_music():
    """Returns a calming music playlist when anxiety is detected."""
    return [
        "https://www.youtube.com/watch?v=2OEL4P1Rz04",  # Deep Sleep Music
        "https://www.youtube.com/watch?v=1ZYbU82GVz4",  # Relaxing Piano
        "https://www.youtube.com/watch?v=3O31iF5c6ac"   # Meditation Music
    ]

music_tool = Tool.from_function(
    func=recommend_calming_music,
    name="calming_music",
    description="Recommends relaxing music links when anxiety is detected."
)

def filter_harmful_response(response: str):
    """Ensure generated response does not contain harmful content."""
    blocked_phrases = ["self-harm", "suicide", "violence"]

    if any(word in response.lower() for word in blocked_phrases):
        return "I'm sorry, but I can't discuss this topic. If you're struggling, please seek help from a licensed professional or helpline."

    return response

guardrail_tool = Tool.from_function(
    func=filter_harmful_response,
    name="guardrail_filter",
    description="Filters AI-generated responses to prevent harmful content."
)


In [ ]:
# from huggingface_hub import login
# login()

In [17]:
import torch
torch.cuda.empty_cache()

In [18]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [19]:
!nvidia-smi

Sat Apr  5 14:35:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             34W /  250W |    4329MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [20]:
!nvidia-smi --query-compute-apps=pid,used_memory --format=csv

pid, used_gpu_memory [MiB]
2910, 4326 MiB


In [21]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline,BitsAndBytesConfig
# from langchain.llms import HuggingFacePipeline
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline 

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Model 1 on CPU
model1_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_memory = AutoModelForCausalLM.from_pretrained(model1_name,torch_dtype=torch.float16,do_sample=False).to(device)
tokenizer_memory = AutoTokenizer.from_pretrained(model1_name)
rephrase_generator = pipeline("text-generation", model=model_memory, tokenizer=tokenizer_memory)
llm_rephrase = HuggingFacePipeline(pipeline=rephrase_generator)

Device set to use cuda:0


In [22]:
# Load Model 2 on GPU
from peft import PeftModel

model2_name = "Jesslyn26/improved_mistral_7b_combined_3"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enables QLoRA (4-bit)
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,  # Use FP16 for computations
    bnb_4bit_use_double_quant=True,  # Double quantization for lower memory
)
mistral_tokenizer = AutoTokenizer.from_pretrained(model2_name)
device = torch.device("cuda:1" if torch.cuda.device_count() > 1 else "cuda:0")
mistral = AutoModelForCausalLM.from_pretrained(model2_name,quantization_config=quantization_config,
    device_map="auto",
    use_cache=False)
mistral_generator = pipeline("text-generation", model=mistral, tokenizer=mistral_tokenizer, max_new_tokens=500) #llm_chain
llm_mistral = HuggingFacePipeline(pipeline=mistral_generator)
chat_model = ChatHuggingFace(llm=llm_mistral)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

...

Device set to use cuda:0


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [23]:
chat_model.invoke("test")

AIMessage(content="<s>[INST] test[/INST] It seems like you'd like to test the functionality of this text-based AI. I'm here to help! How can I assist you today? Let's chat about a topic you're interested in, or ask me a question. I'll do my best to provide a helpful and friendly response.\n\nFor example, you could ask me about:\n- Facts about a specific subject\n- Explanations of complex concepts\n- Recommendations for books, movies, or TV shows\n- Advice on personal or professional issues\n- Instructions for completing a task\n- Or just about anything else you'd like to discuss!\n\nI'm looking forward to our conversation. Let's get started!", additional_kwargs={}, response_metadata={}, id='run-5918c5f6-33be-4d02-8a68-121a55a59549-0')

In [24]:
!nvidia-smi

Sat Apr  5 14:37:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P0             41W /  250W |    8789MiB /  16384MiB |     42%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [25]:
!ps -fp 1729

UID          PID    PPID  C STIME TTY          TIME CMD


In [31]:
from langchain.tools import StructuredTool
from pydantic import BaseModel
from typing import TypedDict
from typing import List, ClassVar
import gc 
from pydantic import Field, root_validator



# Define prompt for generating standalone question
_standalone_template = """
[INST]
You are an empathetic therapy assistant. Given a conversation history and a follow-up user statement,
your task is to rewrite the statement as a **standalone query** to retrieve **relevant therapeutic advice** to give to user.
Chat History:
{chat_history}

Follow Up Input: {question}

Standalone Question:
[/INST]
"""

STANDALONE_QUESTION_PROMPT = PromptTemplate.from_template(_standalone_template)

class RephraseInputDict(BaseModel):
    question: str
    chat_history: str

# Function to rephrase the question (ensure text format for model input)
def rephrase_question(question: str, chat_history: str):
    with torch.no_grad():  
        prompt = STANDALONE_QUESTION_PROMPT.format(question=question, chat_history=chat_history)
        inputs = tokenizer_memory(prompt, return_tensors="pt").to("cuda")
        output = model_memory.generate(**inputs, max_new_tokens=100, temperature=0.0, repetition_penalty=1.1, do_sample=False)
    
    del inputs
    gc.collect()
    torch.cuda.empty_cache() 

    print(tokenizer_memory.decode(output[0], skip_special_tokens=True))
    return tokenizer_memory.decode(output[0], skip_special_tokens=True)

rephrase_tool = StructuredTool.from_function(
    func=rephrase_question,
    name="query_rephraser",
    description="Rephrases user input into a standalone query for FAISS retrieval.",
    args_schema=RephraseInputDict)



# def retrieve_documents(query: str,faiss_index, top_k=4):
#     """Retrieve therapy-related documents based on user input."""
#     query = str(query)
#     documents = faiss_index.similarity_search(query, k=top_k)
#     return [str(doc.page_content) for doc in documents]

# retriever_tool = StructuredTool.from_function(
#     func=retrieve_documents,
#     name="therapy_rag_retrieval",
#     description="Retrieves relevant therapy documents from RAG storage."
# )


# Define function that accepts the schema
def retrieve_documents(input_data):
    """Retrieve therapy-related documents based on user input."""
    documents = faiss_index.similarity_search(input_data, k=4)
    return [doc.page_content for doc in documents]

# Define retriever tool
retriever_tool = StructuredTool.from_function(
    func=retrieve_documents,
    name="therapy_rag_retrieval",
    description="Retrieves relevant therapy documents from FAISS storage.",  
)

In [27]:
from langchain.agents import create_tool_calling_agent
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor
from langchain_core.messages import trim_messages
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langgraph.prebuilt import create_react_agent
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor
from langchain_core.runnables import RunnableLambda

from typing import TypedDict

# Define input/output state schema
class AgentState(TypedDict):
    input: str
    output: str


def generate_rag_response(query,faiss_index, id, rephrase_tool, retriever_tool,llm_mistral):

    safety_flag = suicidal_tool.invoke(query)
    if "crisis" in safety_flag.lower():
        return safety_flag 
    
    memory = get_session_history(id)
    
    chat_history = memory.messages

    chat_history_formatted = "\n".join(map(lambda x:x.content, chat_history))
    
    # #if convo gets long to keep to context window
    # trimmer = trim_messages(
    # max_tokens=1024,
    # strategy="last",
    # token_counter=mistral_tokenizer,
    # include_system=True
    # )

    # print(chat_history)

    # trimmed_history = trimmer.invoke(chat_history)
    
    rephrased_input = rephrase_tool.invoke({"question": query, "chat_history": chat_history_formatted})
    # Retrieve relevant documents from FAISS
    print(rephrased_input)
    retrieved_docs = retriever_tool.invoke(rephrased_input)
    
    context = "\n\n".join(retrieved_docs)

    # Construct prompt for fine-tuned model with context and chat history
    system_prompt_text = f"""
    ### Instruction:
    You are a supportive and empathetic therapy chatbot trained in evidence-based counseling techniques. 

    Guidelines:
    - You **do not diagnose** mental health conditions.
    - You **provide emotional support** and **suggest self-help strategies** based on cognitive behavioral therapy (CBT) and evidence-based practices retrieved from the context.
    - You **detect and address repetitive negative thought patterns** and **gently encourage professional counseling** when needed.
    - You **do not replace a licensed therapist** and will **always refer users to a professional** in cases of severe distress or crisis.
    - You **use non-judgmental, affirming language** that promotes self-compassion and self-awareness.
    
    Response Style:
    - Use **motivational interviewing techniques** (e.g., open-ended questions, reflections, affirmations).
    - Provide **psychoeducation** where relevant.
    - If the user expresses crisis-level distress (e.g., suicidal ideation, self-harm), provide **a crisis intervention message** and direct them to emergency help resources.

    ### Context from Retrieved Documents:
    {context}
    """

    tools=[ memory_tool, suicidal_tool, guardrail_tool, music_tool]
    
    # 2. Create agent using langgraph's create_react_agent
    agent = create_react_agent(
        llm_mistral,           # Your LLM, e.g., HuggingFacePipeline or ChatOpenAI
        tools=tools,               # [memory_tool, suicidal_tool, guardrail_tool, music_tool]
        prompt=system_prompt_text
    )


    def wrapped_agent_node(state):
        prompt = ChatPromptTemplate.from_messages([
            MessagesPlaceholder(variable_name="history"),
            ("human", "{question}"),
        ])
        messages = prompt.invoke(state).to_messages()
        print(messages)
        return agent.invoke({"messages": messages})["messages"][-1]
  
        
    # # 3. Build the graph
    # workflow = StateGraph(dict)  # Could also be StateGraph(dict) for more complex state
    
    # # Add the agent node
    # workflow.add_node("agent",agent)#Runnable(wrapped_agent_node)
    
    # # Set entry point and ending
    # workflow.set_entry_point("agent")
    # workflow.add_edge("agent", END)
    
    # # Compile the graph into a runnable app
    # graph_executor = workflow.compile()

    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ])
    
    # Optional: Add memory tracking using RunnableWithMessageHistory
    graph_with_memory = RunnableWithMessageHistory(
        RunnableLambda(wrapped_agent_node),
        get_session_history,  # your memory/session retriever function
        input_messages_key="question",
        history_messages_key="history",
    )
    
    # 4. Prepare input and invoke
    input_with_id = f"{id}::{query}"
    
    response = graph_with_memory.invoke(
        {"question": input_with_id},
        config={"configurable": {"session_id": id}}
    )
    
    print(response)
    
    final_output = response.content
    
    guard_check = guardrail_tool.invoke(final_output)
    if "flagged" in guard_check.lower():
        # Optionally regenerate or sanitize response
        return "Sorry, I'm unable to provide that response. Let's talk about something safe."


    return final_output

In [35]:
query = "I have been having low self esteem and am struggling to face myself in the mirror"

In [36]:
from langchain.memory import ChatMessageHistory

In [37]:
from langchain.prompts import SystemMessagePromptTemplate
from langchain.agents import initialize_agent
from langchain.agents import AgentType
import uuid
from langchain.memory import ChatMessageHistory

# from langchain.memory.chat_message_histories import InMemoryChatMessageHistory




store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
        print(store[session_id])
    return store[session_id]
    
# # Format chat history list into plain string
# def stringify_chat_history(messages: list) -> str:
#     return "\n".join([f"{m.type}: {m.content}" for m in messages]) if messages else ""

# # Wrap llm_rephrase so it's runnable in LangGraph / LangChain
# llm_rephrase_wrapped = RunnableLambda(lambda state: {
#     "output": llm_rephrase.invoke(stringify_chat_history(state["chat_history"]) + "\n" + state["input"])
# })
                                   
chain = RunnableWithMessageHistory(
    runnable=llm_rephrase,  # your rephrase LLM or chain
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history"
)




def create_session_id():
    return str(uuid.uuid4()) 
id = create_session_id()


response = generate_rag_response(query,faiss_index, id,rephrase_tool, retriever_tool,chat_model)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.




[INST]
You are an empathetic therapy assistant. Given a conversation history and a follow-up user statement,
your task is to rewrite the statement as a **standalone query** to retrieve **relevant therapeutic advice** to give to user.
Chat History:


Follow Up Input: I have been having low self esteem and am struggling to face myself in the mirror

Standalone Question:
[/INST]
</think>

I understand that you're feeling a bit down about your self-esteem, especially when it comes to reflecting on yourself in front of others. It's important to remember that self-esteem isn't just about what someone else sees; it's also about how they feel about themselves. If you'd like, I can help you explore ways to improve your self-esteem or find activities that make you feel more confident. Let me know if you'd like to discuss this further!

[INST]
You are an empathetic therapy assistant. Given a conversation history and a follow-up user statement,
your task is to rewrite the statement as a **standa

In [38]:
response

"<s>[INST] \n    ### Instruction:\n    You are a supportive and empathetic therapy chatbot trained in evidence-based counseling techniques. \n\n    Guidelines:\n    - You **do not diagnose** mental health conditions.\n    - You **provide emotional support** and **suggest self-help strategies** based on cognitive behavioral therapy (CBT) and evidence-based practices retrieved from the context.\n    - You **detect and address repetitive negative thought patterns** and **gently encourage professional counseling** when needed.\n    - You **do not replace a licensed therapist** and will **always refer users to a professional** in cases of severe distress or crisis.\n    - You **use non-judgmental, affirming language** that promotes self-compassion and self-awareness.\n    \n    Response Style:\n    - Use **motivational interviewing techniques** (e.g., open-ended questions, reflections, affirmations).\n    - Provide **psychoeducation** where relevant.\n    - If the user expresses crisis-leve

In [39]:
response = generate_rag_response("Tell me more about the first strategy",faiss_index, id,rephrase_tool, retriever_tool,chat_model)
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



[INST]
You are an empathetic therapy assistant. Given a conversation history and a follow-up user statement,
your task is to rewrite the statement as a **standalone query** to retrieve **relevant therapeutic advice** to give to user.
Chat History:
c60c5944-3394-4719-abb5-5427cb586ad5::I have been having low self esteem and am struggling to face myself in the mirror
<s>[INST] 
    ### Instruction:
    You are a supportive and empathetic therapy chatbot trained in evidence-based counseling techniques. 

    Guidelines:
    - You **do not diagnose** mental health conditions.
    - You **provide emotional support** and **suggest self-help strategies** based on cognitive behavioral therapy (CBT) and evidence-based practices retrieved from the context.
    - You **detect and address repetitive negative thought patterns** and **gently encourage professional counseling** when needed.
    - You **do not replace a licensed therapist** and will **always refer users to a professional** in cases o

System prompt to consider based on therpaist guidelines on holding their session

Deploy with fast api